# Data 1050 SQL Project

## Create Database

In [10]:
pip install mysql-connector-python # type: ignore

Note: you may need to restart the kernel to use updated packages.


In [11]:
import mysql.connector # type: ignore
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password= "7@u*sqJX"   #REPLACE THIS WITH THE PASSWORD YOU SET
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [12]:
#create a database
mycursor = mydb.cursor()
mycursor.execute("DROP DATABASE IF EXISTS data1050SQLProject")
mycursor.execute("CREATE DATABASE data1050SQLProject")

In [13]:
mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('data1050f24',)
('data1050SQLProject',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


### Connecting to a database and showing tables

In [14]:
#connect to a specific database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password= "7@u*sqJX", #REPLACE THIS WITH YOUR PASSWORD 
  database = "data1050SQLProject" #connecting to testDatabase
)

In [15]:
mycursor = mydb.cursor()

mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

In [16]:
import pandas as pd # type: ignore

In [17]:
pip install fsspec # type: ignore

Note: you may need to restart the kernel to use updated packages.


### Add Tables

In [18]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.physicians")
mycursor.execute('''CREATE TABLE data1050SQLProject.physicians ( 
                    ssn VARCHAR(128) PRIMARY KEY, 
                    name VARCHAR(128), 
                    primary_specialty VARCHAR(128),
                    experience_years INT);''')

In [19]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.patients")
mycursor.execute('''CREATE TABLE data1050SQLProject.patients ( 
                    ssn VARCHAR(128) PRIMARY KEY, 
                    name VARCHAR(128), 
                    address VARCHAR(128),
                    birth_date DATETIME,
                    physician_id VARCHAR(128),
                    foreign key(physician_id) references data1050SQLProject.physicians(ssn));''')

In [20]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.pharmacies")
mycursor.execute('''CREATE TABLE data1050SQLProject.pharmacies ( 
                    id INT PRIMARY KEY, 
                    name VARCHAR(128), 
                    address VARCHAR(128),
                    phone VARCHAR(128));''')

In [21]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.drugs")
mycursor.execute('''CREATE TABLE data1050SQLProject.drugs ( 
                    id VARCHAR(128), 
                    name VARCHAR(128) PRIMARY KEY);''')

In [24]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.prescriptions")
mycursor.execute('''CREATE TABLE data1050SQLProject.prescriptions ( 
                    id INT PRIMARY KEY, 
                    patient_id VARCHAR(128), 
                    physician_id VARCHAR(128),
                    drug_name VARCHAR(128),
                    date DATE,
                    quantity INT,
                    foreign key(patient_id) references data1050SQLProject.patients(ssn),
                    foreign key(physician_id) references data1050SQLProject.physicians(ssn),
                    foreign key(drug_name) references data1050SQLProject.drugs(name));''')

In [ ]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.adverse_interactions")
mycursor.execute('''CREATE TABLE data1050SQLProject.adverse_interactions ( 
                    drug_name VARCHAR(128), 
                    drug_name_2 VARCHAR(128), 
                    PRIMARY KEY (drug_name,drug_name_2),
                    foreign key(patient_id) references data1050SQLProject.patients(ssn),
                    foreign key(physician_id) references data1050SQLProject.physicians(ssn),
                    foreign key(drug_name) references data1050SQLProject.drugs(name));''')